# THn visualization with RootInteractive
### Import libraries

* colorcet optional (for more ROOT colz like color pallete import it and use cc.b_linear_kbgoy_20_95_c57)

In [1]:
from datetime import date, datetime
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
import base64
import ctypes
import logging
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
import colorcet as cc

Welcome to JupyROOT 6.24/06
x bokehVisJS3DGraph.ts
x HistogramCDS.ts
Import  CDSCompress.ts
Import  DownsamplerCDS.ts
Import  CDSAlias.ts
Import  CDSJoin.ts


### Load the THnSparse file

In [2]:
myFile = ROOT.TFile.Open("~/Software/TPC_QC_Visualization/Data/UserFiles/DownlodedFiles.root")
hist = myFile.PID
thn = hist.PID
dimensions = ['P', 'dEdxTot','dEdxMax','Phi','Tgl','Snp','NClusters']

### Convert THnSparse to pandas.DataFrame
* the binning parameters should be known (ToDo: investigate if this can be extracted from the THn) 

In [3]:
bin_count = [50,50,50,72,20,20,40]
bin_min = [0,0,0,0,-1.5,-1,0]
bin_max = [10,500,100,6.3,1.5,1,152]
bin_width = [(bin_max[x]-bin_min[x])/(bin_count[x]+1) for x in range(len(bin_count))]

tracks = [];
coord = (ctypes.c_int*7)()
#for Bin in range(thn.GetNbins()):
for Bin in range(10000):
    val = thn.GetBinContent(Bin,coord)
    tracks.append(val)
    for Dim in range(7):
        tracks.append(bin_min[Dim]+(coord[Dim]-0.5)*bin_width[Dim])
    
df = pd.DataFrame(np.reshape(tracks, newshape=(int(len(tracks)/8), 8)), 
                         columns = ['Weight','P', 'dEdxTot','dEdxMax','Phi','Tgl','Snp','NClusters'])
initMetadata(df)
df.head()

,Weight,P,dEdxTot,dEdxMax,Phi,Tgl,Snp,NClusters
0,1.0,0.098039,53.921569,16.666667,0.560959,-0.285714,0.000000,101.951220
1,1.0,0.098039,63.725490,20.588235,5.566438,-0.857143,-0.952381,12.975610
2,4.0,0.490196,44.117647,16.666667,3.495205,-1.142857,0.190476,116.780488
3,10.0,0.294118,53.921569,16.666667,2.114384,-0.857143,0.095238,139.024390
4,1.0,0.098039,53.921569,16.666667,4.530822,-0.285714,-0.095238,31.512195


## Plot as scatter plot
Doesn't make too much sense for us here since the data from all the dimensions are not aggregated

In [4]:
output_file("thn_scatter.html")

figureArray = [
    [['P'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['NClusters'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Phi'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Tgl'], ['dEdxTot'], {"colorZvar": "Weight"}],
    {"size":2, "legend_options": {"label_text_font_size": "13px"}}
]
layout = {
    "dEdx (a.u.) vs p (GeV/c)": [
        [0, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs ncls": [
        [1, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs φ (rad)": [
        [2, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs tanλ": [
        [3, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]]
}
tooltips = []
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'sizing_mode': 'scale_width'}]
    }    
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=layout,
                      tooltips=tooltips, widgetLayout=widgetLayoutDesc)

## As 1D histograms
Makes sense

In [4]:
output_file("thn_1D.html")

histoArray = [
    {"name": "dEdxTot_hist", "variables": ["dEdxTot"], "nbins": 50, "axis": [0, 1], "weights": "Weight"},
    {"name": "NClusters_hist", "variables": ["NClusters"], "nbins": 40, "axis": [1], "weights": "Weight"},
    {"name": "Phi_hist", "variables": ["Phi"], "nbins": 72, "axis": [0, 1], "weights": "Weight"},
    {"name": "Tgl_hist", "variables": ["Tgl"], "nbins": 20, "axis": [0, 1], "weights": "Weight"}
]
figureArray = [
    [['dEdxTot'],['dEdxTot_hist'],{'size': 5}],
    [['NClusters'],['NClusters_hist'],{'size': 5}],
    [['Phi'],['Phi_hist'],{'size': 5}],
    [['Tgl'],['Tgl_hist'],{'size': 5}]
]
figureLayoutDesc=[
    [[0, 2], [1,3], {'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width'}
]
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'width':1}]
    }
tooltips = []
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, plot_width= 500, histogramArray=histoArray)

## As 2D histograms
Also makes sense

In [5]:
output_file("thn_2D.html")

histoArray = [
    {"name": "dEdx_vs_P", "variables": ["P", "dEdxTot"], "nbins": [50, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_ncls", "variables": ["NClusters", "dEdxTot"], "nbins": [40, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_phi", "variables": ["Phi", "dEdxTot"], "nbins": [72, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_tgl", "variables": ["Tgl", "dEdxTot"], "nbins": [20, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []}
]
figureArray = [
    [['P'], ['dEdx_vs_P'],{"visualization_type": "heatmap"}],
    [['NClusters'], ['dEdx_vs_ncls']],
    [['Phi'], ['dEdx_vs_phi']],
    [['Tgl'], ['dEdx_vs_tgl']],
    ["tableHisto", {"rowwise": True}]
]
figureLayoutDesc={
    "dEdx_vs_p" : [[0,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_ncls" : [[1,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_phi" : [[2,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_tgl" : [[3,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}]
}
tooltips = []
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'width':1}]
    }
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, histogramArray=histoArray, plot_width= 500,
                            palette=cc.b_linear_kbgoy_20_95_c57,colors="Bokeh",rescaleColorMapper=True, nCols = 4)